# Makemore - the bigram

## Setup

In [ ]:
!pip install torch
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

## Data discovery

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:8]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words), max(len(w) for w in words)

In [ ]:
# Character level language model is predict predicting next character in the sequence 
# given already some concrete sequence of characters before it

# Every single words in dataset words is a few examples of an input.
# Given the emma => we have .e, em, mm, ma, a.
# We have have following information:
# The words in likely to start with e, 
# then after e there will likely be m,
# after m we might likely have another m,
# after m => a, and at a then words is likely to end

## Bigram Language Model

We are working with 2 characters at a time
We are looking at the 1 character that we are given as an input
and we are trying to predict the next/consecutive one

It's very simple and weak language model, but it's a good place to start :)

### Exploring the bigrams

In [ ]:
btoc = {} # bigrams to count
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2) 
        btoc[bigram] = btoc.get(bigram, 0) + 1

btoc = sorted(btoc.items(), key=lambda x: x[1], reverse=True) # sorting by count

In [ ]:
btoc[:3]

Let's convert the dict into 2 dimensional array. <br>
The Rows -> 1st Char, The Columns -> 2nd Char, of the bigram

In [ ]:
# 26 alphabet letters & one special character - '.' => It means we need 27x27 array to store the bigrams
N = torch.zeros((27,27), dtype=torch.int32) # Input char index - to - Output char index -> CONTAINER for counts

In [ ]:
chars = sorted(list(set(''.join(words) + '.'))) # set of all lower case chars from words + special character '.'

ctoi = {c:i for i,c in enumerate(chars)} # character to index
itoc = {i:c for c,i in ctoi.items()} # index to character

In [ ]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]):
        ix1 = ctoi[ch1]
        ix2 = ctoi[ch2]
        N[ix1, ix2] +=1 # Input char index - to - Output char index - TO -  count

### Visualising the bigram tensor

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range (27):
    for j in range(27):
        chstr = itoc[i] + itoc[j]
        plt.text(j,i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j,i, N[i,j].item(), ha="center", va="top", color='gray')
plt.axis('off');

## Sampling from the model

#### Method: How to sample?

In [ ]:
g = torch.Generator().manual_seed(2147483647) # Will be used to generate an tensor of object in DETERMINISTIC/REPRODUCTIBLE way

p = torch.rand(3, generator=g)
p = p / p.sum() # normalizing to probability (sum up to 1.)
p

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.multinomial.html
# using multinomial to generate samples based on probability (the higher number of samples, the higher accuracy.
num_samples = 10
ix = torch.multinomial(p, num_samples=num_samples, replacement=True, generator=g)
ix

In [ ]:
torch.bincount(ix)

In [ ]:
num_samples = 10000
ix = torch.multinomial(p, num_samples=num_samples, replacement=True, generator=g)
torch.bincount(ix)

## Name generation

In [ ]:
# https://pytorch.org/docs/stable/notes/broadcasting.html

# --- Optimization: Calculating the probabilities outside of the loop ---
P = N.float() 
P /= P.sum(1, keepdim=True); # normalizing the N to Probabilties # arg=1, makes the sum column-wise

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10): # generation of n names; n = 10
    out = [] # container for the chars that will form the NAME
    ix = 0 # starting sampling with the '.'
    while True:
        # --- The below can be optimized (^ Look at the cell above) --- 
        # p = N[ix].float() # get output_chars counts for ix ['.' - in the first run] 
        # p = p / p.sum() # calculating probability
        # --- Using the optimization ---
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() # get one sample based on probability distribution
        out.append(itoc[ix]) # index of the sample to char
        if ix == 0: # if generated '.' -> the name has been generated
            break;
            
    print(''.join(out))

### Checking the model
Sampling from untrained model (Equal probability for all chars).

In [ ]:
for i in range(10): # generation of n names; n = 10
    out = [] # container for the chars that will form the NAME
    ix = 0 # starting sampling with the '.'
    while True:
        p = torch.ones(27) / 27
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() # get one sample based on probability distribution
        out.append(itoc[ix]) # index of the sample to char
        if ix == 0: # if generated '.' -> the name has been generated
            break;
            
    print(''.join(out))

### Result Summary
Generated names look wrong. However, given the simplicty and weakness of the model, It is expected outcome. 
Model performs better then untrained one, which generates even worse "names".

If one would like improve the accuracy. It is adviced to use more advanced mechanisms.

## Evaluation

#### Bit of theory
How to convert the probabilities into single number that measures the quality of the model? <br>

The way to do it is a **Likelihood**! The likelihood is the product of all the probabilities. In good model the product should be as high as possible. <br>Given the values of prob, the product of them will be very small number. So for the convenience, it is adviced to use log_likelihood
>When a, b & c are probs, the **log(a * b * c) = log(a) + log(b) + log(c)**


The highest value of log_likelihood is 0. It would happen If all the probs are 1. Intuitively, the error should be minimized. Therefore, the the log_likelihood is then multiplied by -1, becoming **Negative Log Likelihood** (nnl). Then the NLL needs to be averaged - to become SINGLE metric and can further act as  **loss function**.
<br>

**The goal of the training is to find a params that minimize the loss function (normalized nll)**

In [ ]:
log_likelihood = 0.0
n = 0 # counter for normalization
for w in words: # ['wojciech'] # to check probability of the name
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = ctoi[ch1]
        ix2 = ctoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob) # (l
        log_likelihood += logprob
        n +=1
        #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

In [ ]:
print(f'{log_likelihood=}')

In [ ]:
nll = -log_likelihood
print(f'{nll=}')

In [ ]:
loss = nll / n # normalized (by count - making an average); called normalized nnl or averaged nnl
print(f'{loss=}') # might work as a good loss function

In [ ]:
#GOAL: Maximize likelihood of the data w.r.t. model parameters (statistical modeling)<br>
#equivalent to maximizing the log_likelihood (because log is monotonic)<br>
#equivalent to minimizing the negative log_likelihood<br>
#equivalent to minimizing the average negative log_likelihood<br>

### Model smoothing

In [ ]:
log_likelihood = 0.0
n = 0 # counter for normalization
for w in ['wojciechx']: # to check probability of the name
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = ctoi[ch1]
        ix2 = ctoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob) # (l
        log_likelihood += logprob
        n +=1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

In [ ]:
print(f'{log_likelihood=}')

In [ ]:
nll = -log_likelihood
print(f'{nll=}')

In [ ]:
loss = nll / n # normalized (by count - making an average); called normalized nnl or averaged nnl
print(f'{loss=}') # might work as a good loss function

### Theory
For 'wojciechx', there is no probability to generate the name. It is because the 'hx' has 0 probability. <br>
However, the 'wojciechx' has some probability to occur. For that kind of cases the thing called 'model smoothing' has been introduced.

The goal is to add the number to the ins-to-outs count to add some very small probability for the combinations.


In [ ]:
P = (N+5).float() # Add some value to N, make sure that it won't be 0 probabilities
P /= P.sum(1, keepdim=True);

In [ ]:
log_likelihood = 0.0
n = 0 # counter for normalization
for w in ['wojciechx']: # to check probability of the name
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = ctoi[ch1]
        ix2 = ctoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob) # (l
        log_likelihood += logprob
        n +=1
        print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

In [ ]:
print(f'{log_likelihood=}')

In [ ]:
nll = -log_likelihood
print(f'{nll=}')

In [ ]:
loss = nll / n
print(f'{loss=}') # Model smoothing introduced some probability to 'hx' -> The chances for "Wojciechx" has increased,